Nama: Bening Maulina Fittamami;
email: beningmaulina@gmail.com 

---

Kelas Belajar Pengembangan Machine Learning (Proyek NLP);
Digital Talent Scholarship Kominfo

---


In [1]:
import pandas as pd
import numpy as np
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [3]:
df = pd.read_csv('Corona_NLP_test.csv')
df

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...,...,...,...,...
3793,3794,48746,Israel ??,16-03-2020,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,3795,48747,"Farmington, NM",16-03-2020,Did you panic buy a lot of non-perishable item...,Negative
3795,3796,48748,"Haverford, PA",16-03-2020,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,3797,48749,NaN,16-03-2020,Gov need to do somethings instead of biar je r...,Extremely Negative


In [4]:
df = df.drop(columns=['ScreenName','TweetAt','Location'])
df

,UserName,OriginalTweet,Sentiment
0,1,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,Find out how you can protect yourself and love...,Extremely Positive
3,4,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral
...,...,...,...
3793,3794,Meanwhile In A Supermarket in Israel -- People...,Positive
3794,3795,Did you panic buy a lot of non-perishable item...,Negative
3795,3796,Asst Prof of Economics @cconces was on @NBCPhi...,Neutral
3796,3797,Gov need to do somethings instead of biar je r...,Extremely Negative


In [5]:
df.head()

,UserName,OriginalTweet,Sentiment
0,1,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,Find out how you can protect yourself and love...,Extremely Positive
3,4,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [6]:
category = pd.get_dummies(df.Sentiment)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='Sentiment')
df_baru

,UserName,OriginalTweet,Extremely Negative,Extremely Positive,Negative,Neutral,Positive
0,1,TRENDING: New Yorkers encounter empty supermar...,1,0,0,0,0
1,2,When I couldn't find hand sanitizer at Fred Me...,0,0,0,0,1
2,3,Find out how you can protect yourself and love...,0,1,0,0,0
3,4,#Panic buying hits #NewYork City as anxious sh...,0,0,1,0,0
4,5,#toiletpaper #dunnypaper #coronavirus #coronav...,0,0,0,1,0
...,...,...,...,...,...,...,...
3793,3794,Meanwhile In A Supermarket in Israel -- People...,0,0,0,0,1
3794,3795,Did you panic buy a lot of non-perishable item...,0,0,1,0,0
3795,3796,Asst Prof of Economics @cconces was on @NBCPhi...,0,0,0,1,0
3796,3797,Gov need to do somethings instead of biar je r...,1,0,0,0,0


In [7]:
#One-Hot Encoding
tweet = df_baru['OriginalTweet'].values
label = df_baru[['Extremely Negative','Extremely Positive','Negative','Neutral','Positive']].values

In [18]:
#Membagi Data Training dan Data Testing
from sklearn.model_selection import train_test_split
tweet_latih, tweet_test, label_latih, label_test = train_test_split(tweet, label, test_size=0.2, random_state=123)

In [9]:
len(tweet_test)

760

In [19]:
#Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
     
tokenizer = Tokenizer(num_words=10000, oov_token='x')
tokenizer.fit_on_texts(tweet_latih) 
tokenizer.fit_on_texts(tweet_test)
     
sekuens_latih = tokenizer.texts_to_sequences(tweet_latih)
sekuens_test = tokenizer.texts_to_sequences(tweet_test)
     
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [11]:
padded_test

array([[   0,    0,    0, ...,  259,    8, 5365],
       [   0,    0,    0, ...,   72,  225,   42],
       [   0,    0,    0, ...,   24,  888,    7],
       ...,
       [   0,    0,    0, ...,    9,   11, 1214],
       [   0,    0,    0, ...,   92,   38,  391],
       [   0,    0,    0, ...,    7,  149,  110]], dtype=int32)

In [27]:
#Embedding dan Arsitektur LSTM
import tensorflow as tf
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=10000, output_dim=16),
        tf.keras.layers.LSTM(64),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(5, activation='softmax')
    ])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
model

In [28]:
from tensorflow import keras
class MyCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
       if(logs.get('val_accuracy')>0.75):
        print("\nAkurasi telah mencapai >75%!")
        self.model.stop_training = True

In [39]:
num_epochs = 50
# model.fit(tweet_latih, label_latih, epochs=num_epochs, validation_data=(tweet_test, label_test), verbose=2)
history = model.fit(padded_latih, label_latih, epochs=num_epochs, verbose=2, callbacks=[MyCallback()], validation_split=0.2, validation_data=(padded_test, label_test))

Epoch 1/50
76/76 - 3s - loss: 3.1345e-05 - accuracy: 1.0000 - val_loss: 1.1431e-07 - val_accuracy: 1.0000

Akurasi telah mencapai >75%!
